<a href="https://colab.research.google.com/github/maya70/YHCR/blob/main/BigQuery_bquxjob_47295a2a_18432c95f71.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'yhcr-prd-phm-bia-core' # Project ID inserted based on the query results selected to explore
location = 'europe-west2' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [5]:
def run_sql_query(sql, destination=None):
    """Quick way to run sql queries with bigquery library
    
    Can be used to run sql queries exactly as they would run using the 
    BigQuery SQL Workspace. By setting the "destination" argument, the results
    of a query can be stored as a new table/overwrite an existing table at the
    table id specified.

    Args:
        sql: string, the SQL command to be run
        destination: string (default: None), a table id where the results
            of the SQL command will be stored, if None then results aren't 
            stored

    Returns:
        bigquery.table.Table, containing table object of the stored results of 
            the query if destination argument isn't none
        -- otherwise --
        bigquery.job.queryjob, if no destination is provided and results aren't
            stored

    Example:
    ```python
    # queries example table and stores results in new table
    table = run_sql_query(
        sql = "SELECT * FROM `example.table.id`",
        destination = "destination.for.results"
    )
    
    # caps "value" column at 100
    query = run_sql_query(
        sql = "UPDATE example.table.id SET value = 100 WHERE value > 100"
    )
    ```
    """
    
    if destination:
        job_config = bigquery.QueryJobConfig(
            destination=destination, 
            write_disposition="WRITE_TRUNCATE"
        )
    else:
        job_config=None
    
    query_job = client.query(sql, job_config=job_config)  # Make an API request.
    query_job.result()  # Wait for the job to complete.
    
    if destination:
        result_table = client.get_table(destination)
        return result_table
    else:
        return query_job


In [ ]:
assessment_snomed_codes= ["725882009", "725883004", "710901000000104", "725804008", "170218003", "273396001", "170219006", "273309001" ]
sql_assessment_snomed_list = "(" + ", ".join([f'"{code}' for code in assessment_snomed_codes]) + ")"
def return_yr_date_diff_sql(from_date, to_date, var_name):
    diff_fn = f"DATE_DIFF({to_date}, {from_date}, DAY) / 365.25"
    return f"FLOOR({diff_fn}) AS {var_name}"
age = return_yr_date_diff_sql("demo.DOB_formatted", "CURRENT_DATE()", "age")
age_at_diagnosis = return_yr_date_diff_sql("demo.DOB_formatted", 
                                           "diag.diagnosis_date", 
                                           "age_at_diagnosis")


ethnic_group_regex = "REGEXP_EXTRACT(demo.census_ethnicity, r'^(.+?):')"
ethnic_group = f"""
    CASE
        WHEN {ethnic_group_regex} IS NOT NULL THEN {ethnic_group_regex}
        ELSE "Unknown"
    END AS ethnic_group
"""

ethnic_subgroup_regex = "REGEXP_EXTRACT(demo.census_ethnicity, r':(.+?)-')" 
ethnic_subgroup = f"""
    CASE
        WHEN {ethnic_subgroup_regex} IS NOT NULL THEN {ethnic_subgroup_regex}
        ELSE "Other"
    END AS ethnic_subgroup
"""

sex = """
    CASE
        WHEN demo.remapped_gender = 45766034 THEN "Male"
        WHEN demo.remapped_gender = 45766035 THEN "Female"
        ELSE "Unknown"
    END AS sex
"""
project = "yhcr-prd-phm-bia-core"
srcode_table = f"`{project}.CY_FDM_PrimaryCare_v5.tbl_SRCode`"
#demographics_table = f"`{project}.CY_STAGING_DATABASE.src_DemoGraphics_MASTER`"
lsoas_table = f"`{project}.CY_LOOKUPS.tbl_lsoa_boundaries`"
# build SQL query
sql = f"""
    WITH diag AS (
        SELECT person_id, MIN(src_dateeventrecorded) AS diagnosis_date
        FROM {srcode_table}
        WHERE src_snomedcode IN {sql_asd_snomed_list} 
        GROUP BY person_id
    )
    SELECT diag.*, {age}, {age_at_diagnosis}, {ethnic_group}, 
        {ethnic_subgroup}, {sex}, lsoas.lat_long, lsoas.lsoa_name
    FROM diag
    LEFT JOIN {demographics_table} demo
    ON diag.person_id = demo.person_id
    LEFT JOIN {lsoas_table} lsoas
    ON lsoas.LSOA_code = demo.LSOA
"""

asd_master_tab = f"{project_id}.CY_ASD_data.ASD_master_tab"
query = run_sql_query(sql=sql, destination=asd_master_tab)


## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=yhcr-prd-phm-bia-core:europe-west2:bquxjob_47295a2a_18432c95f71)
back to BigQuery to edit the query within the BigQuery user interface.

In [2]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_47295a2a_18432c95f71') # Job ID inserted based on the query results selected to explore
print(job.query)

select * from `CY_CDM_V1.concept_relationship`
where concept_id_1 in (37111233, 37111232, 3352112, 3269466)
order by concept_id_1



# Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [3]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_47295a2a_18432c95f71') # Job ID inserted based on the query results selected to explore
results = job.to_dataframe()
results

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
0,3269466,37111232,Maps to,1970-01-01,2099-12-31,None
1,3269466,3267911,Has method,2019-08-16,2099-12-31,None
2,3269466,3320021,Is a,2019-08-16,2099-12-31,None
3,3352112,37111233,Maps to,1970-01-01,2099-12-31,None
4,3352112,3320021,Is a,2019-08-16,2099-12-31,None
5,3352112,3267911,Has method,2019-08-16,2099-12-31,None
6,37111232,37111232,Mapped from,2017-07-31,2099-12-31,None
7,37111232,3269466,Mapped from,1970-01-01,2099-12-31,None
8,37111232,4044176,Has method,2017-07-31,2099-12-31,None
9,37111232,507367,Mapped from,2018-01-31,2099-12-31,None


## Show descriptive statistics using describe()
Use the ```pandas DataFrame.describe()```
[method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
to generate descriptive statistics. Descriptive statistics include those that
summarize the central tendency, dispersion and shape of a dataset’s
distribution, excluding ```NaN``` values. You may also use other Python methods
to interact with your data.

In [4]:
results.describe()

,concept_id_1,concept_id_2
count,2.000000e+01,2.000000e+01
mean,2.697110e+07,1.667755e+07
std,1.589171e+07,1.788510e+07
min,3.269466e+06,5.073630e+05
25%,3.352112e+06,3.267911e+06
50%,3.711123e+07,3.698144e+06
75%,3.711123e+07,3.711123e+07
max,3.711123e+07,4.048419e+07
